In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
curated_path_training = f"{data_curated_path}/HC_TRAINING_D"
curated_path_employee = f"{data_curated_path}/HC_EMPLOYEE_D"
curated_path_date = f"{data_curated_path}/COM_DATE_D"

In [0]:
df_employee = spark.read.format("delta").load(curated_path_employee)
df_training = spark.read.format("delta").load(curated_path_training)
df_date = spark.read.format("delta").load(curated_path_date)

In [ ]:
df_employee = df_employee.withColumn('Performance Score', decrypt_data(col('Performance Score'))) \
    .withColumn('Current Employee Rating', decrypt_data(col('Current Employee Rating')))

In [0]:
date_df_monthly = (
    df_date
    .withColumn("SnapshotMonth", F.trunc("date", "month"))
    .select("SnapshotMonth")
    .distinct()
)

In [0]:

df_employee = df_employee.withColumnRenamed("EmpID", "EmployeeID")

employee_snapshots = (
    df_employee.crossJoin(date_df_monthly)
    .filter(
        (F.col("start_date_eff") <= F.col("SnapshotMonth")) &
        ((F.col("end_date_eff").isNull()) | (F.col("end_date_eff") > F.col("SnapshotMonth")))
    )
)

In [0]:
df_training = df_training.withColumnRenamed("Employee ID", "EmployeeID")
train_agg = (
    df_training
    .withColumn("TrainingMonth", F.trunc("Training Date", "month"))
    .groupBy("Employee ID", "TrainingMonth")
    .agg(
        F.count("*").alias("TrainingCount"),
        F.sum("TrainingDuration(Days)").alias("TotalTrainingDays"),
        F.sum("TrainingCost").alias("TotalTrainingCost")
    )
)

In [0]:
fact_snapshot = (
    employee_snapshots
    .join(train_agg, 
          (employee_snapshots.EmployeeID == train_agg.EmployeeID) &
          (employee_snapshots.SnapshotMonth == train_agg.TrainingMonth),
          "left"
    )
    .select(
        "SnapshotMonth",
        "EmployeeID",
        "BusinessUnit",
        "DepartmentType",
        "Division",
        "EmployeeStatus",
        "EmployeeType",
        "EmployeeClassificationType",
        "Performance Score",
        "Current Employee Rating",
        F.coalesce("TrainingCount", F.lit(0)).alias("TrainingCount"),
        F.coalesce("TotalTrainingDays", F.lit(0)).alias("TotalTrainingDays"),
        F.coalesce("TotalTrainingCost", F.lit(0)).alias("TotalTrainingCost")
    )
)

In [ ]:
fact_snapshot = fact_snapshot.withColumn('Performance Score', encrypt_data(col('Performance Score'))) \
    .withColumn('Current Employee Rating', encrypt_data(col('Current Employee Rating')))

In [0]:
integration_id = ['SnapshotMonth', 'EmployeeID']

In [0]:
df_final = update_dw_columns(fact_snapshot , key_column_list, '', 'KAGGLE')
df_final = convert_date_columns(df_final)

In [0]:
write_to_data_lake(df_final, 'delta', 'merge', '{curated_path}/HC_HEADCOUNT_F', integration_id)